# Digit Capcha Detection

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import glob
import cv2

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

In [4]:
MODEL_NAME = 'inference_graph'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'labelmap.pbtxt'

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Detection

In [7]:
def run_inference_for_single_image(image, graph):
    
    if 'detection_masks' in tensor_dict:
        
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
        
        
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
        
    return output_dict

# Load image & Detection

In [8]:
images = glob.glob("test-images/*.jpg")
print(np.shape(images))

(56,)


In [9]:

#now detect digits in each capcha images and save in a dir
for img in images:
    
    #read & resize
    image_np = cv2.imread(img , 1)  #0 => gray  1=>rgb
    
    
    #my input image dim is (135,50) // for beter boxing
    image_np = cv2.resize(image_np, (540, 200)) #540, 200


    try:
        with detection_graph.as_default():
            with tf.Session() as sess:
                
                    # Get handles to input and output tensors
                    ops = tf.get_default_graph().get_operations()
                    all_tensor_names = {output.name for op in ops for output in op.outputs}
                    tensor_dict = {}
                    
                    for key in ['num_detections', 'detection_boxes', 'detection_scores',
                                'detection_classes', 'detection_masks']:
                        
                        tensor_name = key + ':0'
                        
                        if tensor_name in all_tensor_names:
                            tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
                            
                            
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, detection_graph)
                    
                    
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        output_dict['detection_boxes'],
                        output_dict['detection_classes'],
                        output_dict['detection_scores'],
                        category_index,
                        instance_masks=output_dict.get('detection_masks'),
                        use_normalized_coordinates=True,
                        line_thickness=3)
                    
                    
#                     cv2.imshow('img',image_np)
#                     cv2.waitKey(0)
#                     cv2.destroyAllWindows()
                    
                    #save detected image in dir
                    name = os.path.basename(img)
                    print(name)
                    cv2.imwrite("test-result/od_{}".format(name), image_np) 

    

    except Exception as e:
        print(e)
        
    
        
        
print("done !")

13469.jpg
13963.jpg
16723.jpg
18641.jpg
18955.jpg
21118.jpg
21987.jpg
24184.jpg
26971.jpg
27114.jpg
28192.jpg
33261.jpg
34425.jpg
35125.jpg
35165.jpg
36248.jpg
37341.jpg
37826.jpg
37917.jpg
39621.jpg
39872.jpg
42445.jpg
43454.jpg
43675.jpg
44845.jpg
45522.jpg
48446.jpg
53172.jpg
54262.jpg
56687.jpg
58251.jpg
61364.jpg
61787.jpg
65783.jpg
66573.jpg
67213.jpg
67843.jpg
71811.jpg
71896.jpg
74561.jpg
75279.jpg
75672.jpg
75886.jpg
76247.jpg
77573.jpg
78542.jpg
79988.jpg
81251.jpg
82839.jpg
84787.jpg
86111.jpg
87111.jpg
91968.jpg
94551.jpg
99546.jpg
99834.jpg
done !
